### Import

In [14]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, RandomizedSearchCV
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn import neighbors
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif, VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from matplotlib import pyplot


### Load data

In [15]:
def load_data():
    data = pd.read_csv(os.path.join('hn', 'HN_radiomicFeatures.csv'), index_col=0)
    label = data["label"]
    label = label.replace(to_replace={'T12': False, 'T34': True}, value=None)
    data = data.drop(["label"], axis=1)
    feature_names = data.columns.values
    return data, label, feature_names

data, label, feature_names = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


The number of samples: 113
The number of columns: 159


### Check for missing data

In [16]:
# Detect missing values in dataframe
data.isnull().values.any()

False

### Data split

In [17]:
def split_data(data, label):
    train_data, test_data, train_label, test_label = train_test_split(data, label, train_size=0.7)
    return train_data, test_data, train_label, test_label
# train_data, val_data, train_label, val_label = train_test_split(train_val_data, train_val_label, train_size=0.85)

train_data, test_data, train_label, test_label = split_data(data, label)
print(f'The number of train samples: {train_data.shape[0]}')
# print(f'The number of validation samples: {val_data.shape[0]}')
print(f'The number of test samples: {test_data.shape[0]}')

The number of train samples: 79
The number of test samples: 34


### Scaling train data

In [18]:
def scale_data(train_data, test_data, feature_names):
    scaler = StandardScaler().fit(train_data)
    train_data = scaler.transform(train_data)
    test_data = scaler.transform(test_data)
    train_data = pd.DataFrame(train_data, columns=feature_names)
    test_data = pd.DataFrame(test_data, columns=feature_names)
    return train_data, test_data

### Remove features with 0 variance

In [19]:
def remove_zero_var(train_data, test_data):
    selector = VarianceThreshold()
    selector.fit_transform(train_data)
    zero_var_col = [column for column in train_data.columns if column not in train_data.columns[selector.get_support()]]
    train_data.drop(zero_var_col, inplace=True, axis=1)
    test_data.drop(zero_var_col, inplace=True, axis=1)
    return train_data, test_data

### Select best features with Anova test

In [20]:
def select_features(kbest, train_data, train_label, test_data):
    fs = SelectKBest(score_func=f_classif, k=kbest)
    kbest_train = fs.fit_transform(train_data, train_label)
    kbest_test = fs.transform(test_data)
    return kbest_train, kbest_test

### Lasso Regression

In [21]:
def lasso_regression(train_data, train_label, test_data):
    sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
    sel_.fit(train_data, np.ravel(train_label,order='C'))
    sel_.get_support()
    train_data = pd.DataFrame(train_data)
    test_data = pd.DataFrame(test_data)
    removed_feats = train_data.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
    lasso_train = train_data.drop(removed_feats, axis='columns')
    lasso_test = test_data.drop(removed_feats, axis='columns')
    return lasso_train, lasso_test

### Random Forest Classifier

In [24]:
def random_forest(train_data, test_data, train_label, test_label):
    clf = RandomForestClassifier()
    n_estimators = range(20,140,15)
    max_depth = range(8, 15, 2)
    min_samples_leaf = range(2, 7)
    min_samples_split = range(3, 10, 2)

    param_grid = {
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'min_samples_leaf': min_samples_leaf,
        'min_samples_split': min_samples_split
    }

    rf_random = RandomizedSearchCV(estimator=clf, param_distributions=param_grid, n_iter=150, cv=4)
    rf_random.fit(train_data, train_label)
    
    best_params = rf_random.best_params_
    best_model = RandomForestClassifier(n_estimators=best_params['n_estimators'], min_samples_split=best_params['min_samples_split'],
                                    min_samples_leaf=best_params['min_samples_leaf'], max_depth=best_params['max_depth'])
    best_model.fit(train_data,train_label)
    rf_random_train =  best_model.predict(train_data)
    rf_random_test = best_model.predict(test_data)
    score_train = metrics.accuracy_score(train_label, rf_random_train)
    score_test = metrics.accuracy_score(test_label, rf_random_test)
    confusion_matrix_train = metrics.confusion_matrix(train_label, rf_random_train)
    confusion_matrix_test = metrics.confusion_matrix(test_label, rf_random_test)
    return score_train, score_test, confusion_matrix_train, confusion_matrix_test

### Calculate sensitivity, specificity, PPV, NPV

In [25]:
def sens_spec(confusion_matrix):
    true_neg, false_pos, false_neg, true_pos = confusion_matrix.ravel()

    sens = true_pos/(true_pos + false_neg)
    spec = true_neg/(true_neg + false_pos)
    pos_pred_value = true_pos/(true_pos + false_pos)
    neg_pred_value = true_neg/(true_neg + false_neg)
    return sens, spec, pos_pred_value, neg_pred_value

### Find average accuracy over multiple classifications

In [26]:
# Find mean accuracy for kNN classification
loops = 30
score_train_array = np.zeros(loops)
score_test_array = np.zeros(loops)
sens_train_array = np.zeros(loops)
spec_train_array = np.zeros(loops)
pos_pred_value_train_array = np.zeros(loops)
neg_pred_value_train_array = np.zeros(loops)
sens_test_array = np.zeros(loops)
spec_test_array = np.zeros(loops)
pos_pred_value_test_array = np.zeros(loops)
neg_pred_value_test_array = np.zeros(loops)

num_best_features = 80
pca_components = 10
knn_neighbors = 15

for i in range(loops):
    data, label, feature_names = load_data()
    train_data, test_data, train_label, test_label = split_data(data, label)
    train_data, test_data = scale_data(train_data, test_data, feature_names)
    train_data, test_data = remove_zero_var(train_data, test_data)

    # Find best features based on F-scores
    kbest_train, kbest_test = select_features(num_best_features, train_data, train_label, test_data)

    # Lasso regression
    lasso_train, lasso_test = lasso_regression(kbest_train, train_label, kbest_test)

    # Apply random forest classifier
    score_train, score_test, confusion_matrix_train, confusion_matrix_test = random_forest(lasso_train, lasso_test, train_label, test_label)
    score_train_array[i] = score_train
    score_test_array[i] = score_test

    sens_train, spec_train, pos_pred_value_train, neg_pred_value_train = sens_spec(confusion_matrix_train)
    sens_test, spec_test, pos_pred_value_test, neg_pred_value_test = sens_spec(confusion_matrix_test)

    sens_train_array[i] = sens_train
    spec_train_array[i] = spec_train
    pos_pred_value_train_array[i] = pos_pred_value_train
    neg_pred_value_train_array[i] = neg_pred_value_train

    sens_test_array[i] = sens_test
    spec_test_array[i] = spec_test
    pos_pred_value_test_array[i] = pos_pred_value_test
    neg_pred_value_test_array[i] = neg_pred_value_test


mean_score_train = np.mean(score_train_array)
mean_score_test = np.mean(score_test_array)
mean_sens_train = np.mean(sens_train)
mean_spec_train = np.mean(spec_train)
mean_ppv_train = np.mean(pos_pred_value_train)
mean_npv_train = np.mean(neg_pred_value_train)
mean_sens_test = np.mean(sens_test)
mean_spec_test = np.mean(spec_test)
mean_ppv_test = np.mean(pos_pred_value_test)
mean_npv_test = np.mean(neg_pred_value_test)

print(f"Mean training score: {mean_score_train}")
print(f"Mean test score: {mean_score_test}")
print(f"Mean train sens: {mean_sens_train}")
print(f"Mean test sens: {mean_sens_test}")
print(f"Mean train spec: {mean_spec_train}")
print(f"Mean test spec: {mean_spec_test}")
print(f"Mean train PPV {mean_ppv_train}")
print(f"Mean test PPV: {mean_ppv_test}")
print(f"Mean train NPV: {mean_npv_train}")
print(f"Mean test NPV: {mean_npv_test}")

Mean training score: 0.9405063291139241
Mean test score: 0.7372549019607845
Mean train sens: 0.8611111111111112
Mean test sens: 0.6842105263157895
Mean train spec: 0.9767441860465116
Mean test spec: 0.8666666666666667
Mean train PPV 0.96875
Mean test PPV: 0.8666666666666667
Mean train NPV: 0.8936170212765957
Mean test NPV: 0.6842105263157895
